<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>Logistic Regression with Keras</H1></u></center>

In [1]:
import tensorflow.compat.v1 as tf
tf.enable_eager_execution()
from keras.backend.tensorflow_backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                
sess = tf.Session(config=config)
tf.disable_v2_behavior()
tf.compat.v1.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1



In [2]:

from keras.datasets import mnist

In [3]:
tf.compat.v1.executing_eagerly()

False

## Preprocessing the data:

In [4]:
# the data, shuffled and split between train and test sets 
(X_train_, y_train), (X_test_, y_test) = mnist.load_data()

In [5]:
input_dim = 784 #28*28 
X_train = X_train_.reshape(60000, input_dim) 
X_test = X_test_.reshape(10000, input_dim) 
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 
X_train /= 255 
X_test /= 255

In [6]:
from keras.utils import np_utils
n_classes = len(set(y_train))
Y_train = np_utils.to_categorical(y_train, n_classes) 
Y_test = np_utils.to_categorical(y_test, n_classes)

## Creating the model:

In [7]:
from keras.models import Sequential 
from keras.layers import Dense, Activation 

In [8]:
output_dim = n_classes = 10 
model = Sequential() 
model.add(Dense(output_dim, input_dim=input_dim, activation='softmax')) 
batch_size = 128 
n_epoch = 20

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## Compile the model:

In [9]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 

## Train the model:

In [10]:
#verbose: Integer. 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
#model.fit(X_train, Y_train, batch_size=batch_size, epochs=n_epoch,verbose=1, validation_data=(X_test, Y_test)) 

## Evaluate the model:

In [11]:
score = model.evaluate(X_test, Y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 2.4537977977752687
Test accuracy: 0.10679999738931656


## CallBacks:

In [12]:
import keras

In [13]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [14]:
history = LossHistory()

In [15]:
model.fit(X_train, Y_train, batch_size=128, epochs=5, verbose=1, validation_data=(X_test, Y_test), callbacks=[history])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 1s 13us/step - loss: 1.2935 - accuracy: 0.6874 - val_loss: 0.8170 - val_accuracy: 0.8299
Epoch 2/5
60000/60000 [==============================] - 1s 13us/step - loss: 0.7219 - accuracy: 0.8379 - val_loss: 0.6098 - val_accuracy: 0.8605
Epoch 3/5
60000/60000 [==============================] - 1s 14us/step - loss: 0.5900 - accuracy: 0.8592 - val_loss: 0.5266 - val_accuracy: 0.8726
Epoch 4/5
60000/60000 [==============================] - 1s 14us/step - loss: 0.5267 - accuracy: 0.8699 - val_loss: 0.4802 - val_accuracy: 0.8819
Epoch 5/5
60000/60000 [==============================] - 1s 13us/step - loss: 0.4884 - accuracy: 0.8764 - val_loss: 0.4501 - val_accuracy: 0.8878


## Save the model and weights:

In [16]:
model.save_weights('../data/mnist_lr_weights.h5', overwrite=True)

In [17]:

model.save('../data/mnist_lr.h5')

NotImplementedError: numpy() is only available when eager execution is enabled.

In [ ]:
from keras.models import load_model

In [ ]:
loaded_model = load_model('../data/mnist_lr.h5')

In [ ]:
preds = loaded_model.predict(X_test)

In [ ]:
preds[0]

## Printing numbers and predictions:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
n = 10
plt.figure(figsize=(15,15))
for i in range(n):
    plt.subplot(1, n, i + 1)
    plt.imshow(X_test_[i], cmap='gray')
    plt.title("Label: {}\nPredicted: {}".format(y_test[i], np.argmax(preds[i])))
    plt.axis('off')
plt.show()

## Reference:

https://keras.io/callbacks/

https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model